<a href="https://colab.research.google.com/github/godineanvesh/Movie_recommender_system/blob/main/Advanced_movie_recommender_system_using_content_based_filtering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

In [2]:
movies = pd.read_csv("https://github.com/SavioSal/datasets/raw/master/movie_clean.csv", encoding = 'unicode_escape')
movies.head()

,title,genres,runtime,vote_average,vote_count,year,overview
0,Toy Story,"['animation', 'comedy', 'family']",81.0,7.7,5415.0,1995.0,"Led by Woody, Andy's toys live happily in his ..."
1,Jumanji,"['adventure', 'fantasy', 'family']",104.0,6.9,2413.0,1995.0,When siblings Judy and Peter discover an encha...
2,Grumpier Old Men,"['romance', 'comedy']",101.0,6.5,92.0,1995.0,A family wedding reignites the ancient feud be...
3,Waiting to Exhale,"['comedy', 'drama', 'romance']",127.0,6.1,34.0,1995.0,"Cheated on, mistreated and stepped on, the wom..."
4,Father of the Bride Part II,['comedy'],106.0,5.7,173.0,1995.0,Just when George Banks has recovered from his ...


In [3]:
#vectorizing #term frequency inverse document frequency
from sklearn.feature_extraction.text import TfidfVectorizer

In [4]:
tfidf = TfidfVectorizer(stop_words = 'english')


In [9]:
movie_new = movies[movies['vote_average'] > 7]

In [11]:
movie_new.head()

,title,genres,runtime,vote_average,vote_count,year,overview
0,Toy Story,"['animation', 'comedy', 'family']",81.0,7.7,5415.0,1995.0,"Led by Woody, Andy's toys live happily in his ..."
5,Heat,"['action', 'crime', 'drama', 'thriller']",170.0,7.7,1886.0,1995.0,"Obsessive master thief, Neil McCauley leads a ..."
12,Balto,"['family', 'animation', 'adventure']",78.0,7.1,423.0,1995.0,An outcast half-wolf risks his life to prevent...
13,Nixon,"['history', 'drama']",192.0,7.1,72.0,1995.0,An all-star cast powers this epic look at Amer...
15,Casino,"['drama', 'crime']",178.0,7.8,1343.0,1995.0,The life of the gambling paradise â Las Vega...


In [13]:
movie_new = movie_new.reset_index()

In [14]:
movie_new.head()

,level_0,index,title,genres,runtime,vote_average,vote_count,year,overview
0,0,0,Toy Story,"['animation', 'comedy', 'family']",81.0,7.7,5415.0,1995.0,"Led by Woody, Andy's toys live happily in his ..."
1,1,5,Heat,"['action', 'crime', 'drama', 'thriller']",170.0,7.7,1886.0,1995.0,"Obsessive master thief, Neil McCauley leads a ..."
2,2,12,Balto,"['family', 'animation', 'adventure']",78.0,7.1,423.0,1995.0,An outcast half-wolf risks his life to prevent...
3,3,13,Nixon,"['history', 'drama']",192.0,7.1,72.0,1995.0,An all-star cast powers this epic look at Amer...
4,4,15,Casino,"['drama', 'crime']",178.0,7.8,1343.0,1995.0,The life of the gambling paradise â Las Vega...


In [17]:
len(movie_new[movie_new['overview'].isna()])

156

In [20]:
movie_new['overview'] = movie_new['overview'].fillna("")

In [22]:
tfidf_matrix = tfidf.fit_transform(movie_new['overview'])

In [26]:
print('tfidf_matrix', tfidf_matrix)

tfidf_matrix   (0, 7817)	0.14272795631004825
  (0, 1967)	0.16153122597090963
  (0, 16185)	0.10595912814741203
  (0, 9678)	0.11097031197711084
  (0, 8669)	0.1373439598965242
  (0, 20281)	0.10929419108108458
  (0, 25058)	0.1373439598965242
  (0, 5327)	0.1306659428259581
  (0, 21348)	0.14172672975415035
  (0, 12734)	0.1075589826875022
  (0, 21260)	0.09652903133794946
  (0, 16780)	0.12750260536071006
  (0, 878)	0.150047309129906
  (0, 24664)	0.11885885279058876
  (0, 16449)	0.18997947475779023
  (0, 4209)	0.48459367791272895
  (0, 3888)	0.11255836940637114
  (0, 3231)	0.12701973428860708
  (0, 24016)	0.11506404784924885
  (0, 12512)	0.14489006721939837
  (0, 16587)	0.0931508115561448
  (0, 28621)	0.14730710157746935
  (0, 1434)	0.4251801892624511
  (0, 30910)	0.459631939785462
  (0, 16208)	0.1140247840744468
  :	:
  (7713, 4340)	0.1503089312027454
  (7713, 12508)	0.09888574941523143
  (7713, 10629)	0.11726809869346727
  (7713, 19081)	0.11178752622335994
  (7713, 22773)	0.11896747869450994


In [27]:
from sklearn.metrics.pairwise import linear_kernel

In [28]:
cosine_similarity = linear_kernel(tfidf_matrix, tfidf_matrix)

In [29]:
cosine_similarity

array([[1.        , 0.        , 0.        , ..., 0.        , 0.00434795,
        0.        ],
       [0.        , 1.        , 0.        , ..., 0.        , 0.        ,
        0.01600189],
       [0.        , 0.        , 1.        , ..., 0.        , 0.00382082,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 1.        , 0.        ,
        0.00577313],
       [0.00434795, 0.        , 0.00382082, ..., 0.        , 1.        ,
        0.00922048],
       [0.        , 0.01600189, 0.        , ..., 0.00577313, 0.00922048,
        1.        ]])

In [30]:
indices = pd.Series(movie_new.index, index = movie_new['title']).drop_duplicates()

In [32]:
indices.head()

title
Toy Story    0
Heat         1
Balto        2
Nixon        3
Casino       4
dtype: int64

In [36]:
def movie_recommender(title):
  idx = indices[title]
  similarity_scores = list(enumerate(cosine_similarity[idx]))
  similarity_scores = sorted(similarity_scores,key = lambda x:x[1],reverse = True)
  scores_final = similarity_scores[1:11]
  movie_indices = [i[0] for i in scores_final]
  return movie_new['title'].iloc[movie_indices]

In [42]:
user_movie = input('please enter your movie:')
movie_recommender(user_movie)

please enter your movie:Toy Story


3029    Casino Jack and the United States of Money
7267                                     4.1 Miles
235                       A Streetcar Named Desire
3371                            X-Men: First Class
2384                          The Savage Innocents
5081                               Execution Squad
753                                  Man Bites Dog
3993                              What's in a Name
474                                    Poltergeist
3230          Russell Peters: Red, White and Brown
Name: title, dtype: object

In [46]:
similarity_scores = list(enumerate(cosine_similarity[1]))
similarity_scores = sorted(similarity_scores,key = lambda x:x[1],reverse = True)
scores_final = similarity_scores[1:11]

In [47]:
scores_final

[(3450, 0.2064131249548036),
 (4038, 0.14022508759108776),
 (7589, 0.13890401872824942),
 (5066, 0.1250964950646683),
 (1882, 0.11530001321198101),
 (6770, 0.11358324200144546),
 (3921, 0.103185830005108),
 (2091, 0.10170290537338701),
 (6829, 0.10125331379356632),
 (5436, 0.09962161535698474)]